
<br>
===============<br>
Radon transform<br>
===============<br>
In computed tomography, the tomography reconstruction problem is to obtain<br>
a tomographic slice image from a set of projections [1]_. A projection is<br>
formed by drawing a set of parallel rays through the 2D object of interest,<br>
assigning the integral of the object's contrast along each ray to a single<br>
pixel in the projection. A single projection of a 2D object is one dimensional.<br>
To enable computed tomography reconstruction of the object, several projections<br>
must be acquired, each of them corresponding to a different angle between the<br>
rays with respect to the object. A collection of projections at several angles<br>
is called a sinogram, which is a linear transform of the original image.<br>
The inverse Radon transform is used in computed tomography to reconstruct<br>
a 2D image from the measured projections (the sinogram). A practical, exact<br>
implementation of the inverse Radon transform does not exist, but there are<br>
several good approximate algorithms available.<br>
As the inverse Radon transform reconstructs the object from a set of<br>
projections, the (forward) Radon transform can be used to simulate a<br>
tomography experiment.<br>
This script performs the Radon transform to simulate a tomography experiment<br>
and reconstructs the input image based on the resulting sinogram formed by<br>
the simulation. Two methods for performing the inverse Radon transform<br>
and reconstructing the original image are compared: The Filtered Back<br>
Projection (FBP) and the Simultaneous Algebraic Reconstruction<br>
Technique (SART).<br>
For further information on tomographic reconstruction, see:<br>
.. [1] AC Kak, M Slaney, "Principles of Computerized Tomographic Imaging",<br>
       IEEE Press 1988. http://www.slaney.org/pct/pct-toc.html<br>
.. [2] Wikipedia, Radon transform,<br>
       https://en.wikipedia.org/wiki/Radon_transform#Relationship_with_the_Fourier_transform<br>
.. [3] S Kaczmarz, "Angenaeherte Aufloesung von Systemen linearer<br>
       Gleichungen", Bulletin International de l'Academie Polonaise<br>
       des Sciences et des Lettres, 35 pp 355--357 (1937)<br>
.. [4] AH Andersen, AC Kak, "Simultaneous algebraic reconstruction<br>
       technique (SART): a superior implementation of the ART algorithm",<br>
       Ultrasonic Imaging 6 pp 81--94 (1984)<br>
The forward transform<br>
=====================<br>
As our original image, we will use the Shepp-Logan phantom. When calculating<br>
the Radon transform, we need to decide how many projection angles we wish<br>
to use. As a rule of thumb, the number of projections should be about the<br>
same as the number of pixels there are across the object (to see why this<br>
is so, consider how many unknown pixel values must be determined in the<br>
reconstruction process and compare this to the number of measurements<br>
provided by the projections), and we follow that rule here. Below is the<br>
original image and its Radon transform, often known as its *sinogram*:<br>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale

In [ ]:
image = shepp_logan_phantom()
image = rescale(image, scale=0.4, mode='reflect', channel_axis=None)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))

In [ ]:
ax1.set_title("Original")
ax1.imshow(image, cmap=plt.cm.Greys_r)

In [ ]:
theta = np.linspace(0., 180., max(image.shape), endpoint=False)
sinogram = radon(image, theta=theta)
dx, dy = 0.5 * 180.0 / max(image.shape), 0.5 / sinogram.shape[0]
ax2.set_title("Radon transform\n(Sinogram)")
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax2.imshow(sinogram, cmap=plt.cm.Greys_r,
           extent=(-dx, 180.0 + dx, -dy, sinogram.shape[0] + dy),
           aspect='auto')

In [ ]:
fig.tight_layout()
plt.show()

####################################################################<br>
<br>
Reconstruction with the Filtered Back Projection (FBP)<br>
======================================================<br>
<br>
The mathematical foundation of the filtered back projection is the Fourier<br>
slice theorem [2]_. It uses Fourier transform of the projection and<br>
interpolation in Fourier space to obtain the 2D Fourier transform of the<br>
image, which is then inverted to form the reconstructed image. The filtered<br>
back projection is among the fastest methods of performing the inverse<br>
Radon transform. The only tunable parameter for the FBP is the filter,<br>
which is applied to the Fourier transformed projections. It may be used to<br>
suppress high frequency noise in the reconstruction. ``skimage`` provides<br>
the filters 'ramp', 'shepp-logan', 'cosine', 'hamming', and 'hann':

In [ ]:
import matplotlib.pyplot as plt
from skimage.transform.radon_transform import _get_fourier_filter

In [ ]:
filters = ['ramp', 'shepp-logan', 'cosine', 'hamming', 'hann']

In [ ]:
for ix, f in enumerate(filters):
    response = _get_fourier_filter(2000, f)
    plt.plot(response, label=f)

In [ ]:
plt.xlim([0, 1000])
plt.xlabel('frequency')
plt.legend()
plt.show()

####################################################################<br>
Applying the inverse radon transformation with the 'ramp' filter, we get:

In [ ]:
from skimage.transform import iradon

In [ ]:
reconstruction_fbp = iradon(sinogram, theta=theta, filter_name='ramp')
error = reconstruction_fbp - image
print(f"FBP rms reconstruction error: {np.sqrt(np.mean(error**2)):.3g}")

In [ ]:
imkwargs = dict(vmin=-0.2, vmax=0.2)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5),
                               sharex=True, sharey=True)
ax1.set_title("Reconstruction\nFiltered back projection")
ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.set_title("Reconstruction error\nFiltered back projection")
ax2.imshow(reconstruction_fbp - image, cmap=plt.cm.Greys_r, **imkwargs)
plt.show()

####################################################################<br>
<br>
Reconstruction with the Simultaneous Algebraic Reconstruction Technique<br>
=======================================================================<br>
<br>
Algebraic reconstruction techniques for tomography are based on a<br>
straightforward idea: for a pixelated image the value of a single ray in a<br>
particular projection is simply a sum of all the pixels the ray passes<br>
through on its way through the object. This is a way of expressing the<br>
forward Radon transform. The inverse Radon transform can then be formulated<br>
as a (large) set of linear equations. As each ray passes through a small<br>
fraction of the pixels in the image, this set of equations is sparse,<br>
allowing iterative solvers for sparse linear systems to tackle the system<br>
of equations. One iterative method has been particularly popular, namely<br>
Kaczmarz' method [3]_, which has the property that the solution will<br>
approach a least-squares solution of the equation set.<br>
<br>
The combination of the formulation of the reconstruction problem as a set<br>
of linear equations and an iterative solver makes algebraic techniques<br>
relatively flexible, hence some forms of prior knowledge can be<br>
incorporated with relative ease.<br>
<br>
``skimage`` provides one of the more popular variations of the algebraic<br>
reconstruction techniques: the Simultaneous Algebraic Reconstruction<br>
Technique (SART) [4]_. It uses Kaczmarz' method as the iterative<br>
solver. A good reconstruction is normally obtained in a single iteration,<br>
making the method computationally effective. Running one or more extra<br>
iterations will normally improve the reconstruction of sharp, high<br>
frequency features and reduce the mean squared error at the expense of<br>
increased high frequency noise (the user will need to decide on what number<br>
of iterations is best suited to the problem at hand. The implementation in<br>
``skimage`` allows prior information of the form of a lower and upper<br>
threshold on the reconstructed values to be supplied to the reconstruction.

In [ ]:
from skimage.transform import iradon_sart

In [ ]:
reconstruction_sart = iradon_sart(sinogram, theta=theta)
error = reconstruction_sart - image
print("SART (1 iteration) rms reconstruction error: "
      f"{np.sqrt(np.mean(error**2)):.3g}")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8.5), sharex=True, sharey=True)
ax = axes.ravel()

In [ ]:
ax[0].set_title("Reconstruction\nSART")
ax[0].imshow(reconstruction_sart, cmap=plt.cm.Greys_r)

In [ ]:
ax[1].set_title("Reconstruction error\nSART")
ax[1].imshow(reconstruction_sart - image, cmap=plt.cm.Greys_r, **imkwargs)

Run a second iteration of SART by supplying the reconstruction<br>
from the first iteration as an initial estimate

In [ ]:
reconstruction_sart2 = iradon_sart(sinogram, theta=theta,
                                   image=reconstruction_sart)
error = reconstruction_sart2 - image
print("SART (2 iterations) rms reconstruction error: "
      f"{np.sqrt(np.mean(error**2)):.3g}")

In [ ]:
ax[2].set_title("Reconstruction\nSART, 2 iterations")
ax[2].imshow(reconstruction_sart2, cmap=plt.cm.Greys_r)

In [ ]:
ax[3].set_title("Reconstruction error\nSART, 2 iterations")
ax[3].imshow(reconstruction_sart2 - image, cmap=plt.cm.Greys_r, **imkwargs)
plt.show()